<a href="https://colab.research.google.com/github/Lynnluxu/tpRNN/blob/master/LSTM_training_notebook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [0]:
##Load an MNIST dataset with a tensor transformation
train_dataset = dsets.##(root='./data', 
                            train=True, 
                            transform=##,
                            download=True)

test_dataset = dsets.##(root='./data', 
                           train=False, 
                           transform=##)

In [0]:
#print train data size
print(train_dataset.train_data.size())

In [0]:
##print train labels size
print(train_dataset.##)

In [0]:
##print test data size
print(test_dataset.##)

In [0]:
##print test labels size
print(test_dataset.##)

In [0]:
#iterate on the dataset
batch_size = 100
n_iters = 3000
## caluculate num_epochs knwing that (number of epochs = number of iterations / (dataset lenght/ batch size))
num_epochs = ##
num_epochs = int(num_epochs)

##use torch utils dataloader to load the training and test data
train_loader = torch.utils.data.##(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.##(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
#Creating an LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your LSTM
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        ## add grad in the hidden state and the cell state initialization
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).##

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).##
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        ##detach h0 and c0
        out, (hn, cn) = self.lstm(x, (##(), ##()))

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

In [0]:
input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10

In [0]:
##instantiate model class
model = ##

In [0]:
##Choose a loss function knowing that it's a classification problem)
criterion = nn.##

In [0]:
##Choose a learning rate then instantiate it to the optimizer)
learning_rate = ##

optimizer = torch.optim.SGD(model.parameters(), lr=##)  

In [0]:
seq_dim = 28  

##iniate the number of iterations (iter) read the loops below to understand
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as a torch tensor with gradient accumulation abilities
        images = images.view(-1, seq_dim, input_dim).requires_grad_()

        ## Clear gradients 
        optimizer.##

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        ## add the labels to the loss
        loss = criterion(outputs, ##)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Resize images
                images = images.view(-1, seq_dim, input_dim)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))